In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.alert import Alert
import time
import traceback
import pandas as pd
import urllib.request
from pandas import DataFrame

In [2]:
pill_names = []
pill_images = []
pill_codes = []
pill_company = ['유한양행', '녹십자', '광동제약', '대웅', '한미약품', '셀트리온', '종근당', '제일약품', '동아에스티', '중외제약', '일동제약', '보령제약', '한독', '동국제약', '휴온스']

In [3]:
main_url = 'https://www.health.kr/searchIdentity/search.asp'

for i in pill_company:
    print('=' * 100)
    print('[회사:', i, ']')
    
    driver = webdriver.Chrome(executable_path="C:/python_edu/program/chromedriver.exe")
    driver.get(main_url)
    time.sleep(1)
    driver.find_element_by_css_selector('#type_01 > a').click()
    driver.find_element_by_css_selector('#type_02 > a').click()
    driver.find_element_by_css_selector('#type_03 > a').click()
    driver.find_element_by_id('firm_name').send_keys(i)
    driver.find_element_by_css_selector('#btn_idfysearch').click()
    time.sleep(1)
    driver.find_element_by_css_selector('#result > ul > li:nth-child(2)').click()
    time.sleep(1)
    
    totalCount = driver.find_element_by_css_selector('#idfy_total_PY_view')
    print('검색 결과:', totalCount.text)
    totalCount = int(totalCount.text.replace(',', ''))
    totalPage = (totalCount // 50) + 1
    print('검색 페이지:', totalPage)
    
    for page in range(1, totalPage + 1):
        try:
            driver.get(main_url)
            time.sleep(1)
            
            driver.find_element_by_css_selector('#type_01 > a').click()
            driver.find_element_by_css_selector('#type_02 > a').click()
            driver.find_element_by_css_selector('#type_03 > a').click()
            driver.find_element_by_id('firm_name').send_keys(i)
            driver.find_element_by_css_selector('#btn_idfysearch').click()
            time.sleep(2)
            
            driver.find_element_by_css_selector('#result > ul > li:nth-child(2)').click()
            time.sleep(1)
            
            driver.execute_script('javascript:changePage(%s)'% page)
            time.sleep(1)
            print('%s 페이지 이동' % page)
            web_url = driver.page_source
            soup = BeautifulSoup(web_url, 'html.parser')
            t1 = soup.find('table',{'class':'tab3_cont list_basic bt0'})
            boxItems = t1.find_all('tr')
            
            for tr in boxItems[2::4]:
                try:
                    image = tr.find('div',{'class' : 'infoOver view_drugimg'}).find('a')
                    pill_image = image.find('img')['src']
                    content = tr.find('td',{'class':'txtL name'})
                    name = content.text.split('\n',1)[0]
                    name_nospace = name.replace('\n', '')
                    pill_name = name_nospace.replace('/','')
                    html_code = content['onclick'].split(',',1)[1]
                    pill_html_code = html_code.replace("'",'').replace(')','')

                    try:
                        pill_url = 'https://www.health.kr/searchDrug/result_drug.asp?drug_cd=%s' % pill_html_code
                        driver.get(pill_url)
                        
                        try:
                            Alert(driver).accept()
                            print("alert accepted")
                            pill_code = pill_name
                        except:
                            print("no alert")

                    # driver.switchTo().alert().accept();
                            html = driver.page_source
                            soup = BeautifulSoup(html, 'html.parser')
                            table = soup.find('table', {'class': 'infotable'})
                            trs = table.find_all('tr')
                    # print(trs[12])
                            pill_code = trs[12].find('span', {'style': 'width:60px'}).text
                    # pill_codes.append(pill_code)
                            time.sleep(1)
                    except:
                        pass
                        
                    pill_images.append(pill_image)
                    pill_names.append(pill_name)
                    pill_codes.append(pill_code)
                    urllib.request.urlretrieve(pill_image, 'img_train/' + pill_code + '.jpg')

                except Exception as e:
                    print('오류', e)
        except Exception as e1:
            print('오류', e1)
            
    driver.close()

[회사: 유한양행 ]
검색 결과: 244
검색 페이지: 5
오류 Message: unknown error: Element <a onclick="type_clicks('정제');">...</a> is not clickable at point (153, 23). Other element would receive the click: <a href="/searchIdentity/search.asp" on="" click="beforeLoadChk(this);return false;">...</a>
  (Session info: chrome=76.0.3809.100)
  (Driver info: chromedriver=2.42.591088 (7b2b2dca23cca0862f674758c9a3933e685c27d5),platform=Windows NT 10.0.17763 x86_64)

오류 Message: unknown error: Element <a onclick="type_clicks('정제');">...</a> is not clickable at point (153, 23). Other element would receive the click: <a href="/searchIdentity/search.asp" on="" click="beforeLoadChk(this);return false;">...</a>
  (Session info: chrome=76.0.3809.100)
  (Driver info: chromedriver=2.42.591088 (7b2b2dca23cca0862f674758c9a3933e685c27d5),platform=Windows NT 10.0.17763 x86_64)

오류 Message: unknown error: Element <a onclick="type_clicks('정제');">...</a> is not clickable at point (153, 23). Other element would receive the click: <a

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#result > ul > li:nth-child(2)"}
  (Session info: chrome=76.0.3809.100)
  (Driver info: chromedriver=2.42.591088 (7b2b2dca23cca0862f674758c9a3933e685c27d5),platform=Windows NT 10.0.17763 x86_64)


In [ ]:
data = {'pill_name' : pill_name, 'pill_code' : pill_code, 'pill_link' : pill_link}

pill_Data = pd.DataFrame(data)